###  실습 1: LangChain 한글 요약 Agent

In [6]:
!pip install langchain_community langchain_openai
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.8 MB/s eta 

In [2]:
my_api_key = ""

In [3]:
import os
os.environ["OPENAI_API_KEY"] = my_api_key

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA


In [4]:

loader = TextLoader("./학교규칙_예제.txt", encoding="utf-8")
documents = loader.load()
print(documents[0])


page_content='title	content
학생 휴대전화 사용 규정	수업 중에는 학생의 휴대전화 사용이 금지되며, 수업 외 시간에만 사용 가능합니다.
교복 착용 규정	학교 지정 교복을 매일 착용해야 하며, 체육수업이 있는 날에는 체육복으로 등교할 수 있습니다.
2025학년도 2학기 학사일정	9월 1일 개학, 9월 5일 체육대회, 9월 15일부터 19일까지 중간고사, 12월 20일 종업식 예정입니다.
학부모 상담 기록 보관	학부모 상담 기록은 상담일로부터 5년간 보관하며, 이후 폐기합니다.
체육대회 일정	2025년 9월 5일(금) 오전 9시부터 학교 운동장에서 진행됩니다. 이어달리기, 피구, 축구가 진행되며 학부모 참여 가능.
중간고사 일정	2025년 9월 15일부터 19일까지 2학기 중간고사가 진행됩니다.
학급 청소 규칙	점심시간 이후 10분간 각자 담당 구역을 청소하며, 청소 당번은 청소 상태를 확인합니다.
수업 준비물 규칙	모든 학생은 매 수업 시간에 교과서, 공책, 필기도구를 반드시 지참해야 합니다.
숙제 제출 규칙	숙제는 수업 후 다음날 아침 조회시간 전까지 제출해야 하며, 지각 제출 시 지각으로 처리됩니다.
시험 일정 안내	이번 학기 중간고사는 10월 15일부터 10월 19일까지 진행되며, 기말고사는 12월 10일부터 12월 14일까지입니다.
체육복 착용 규칙	체육 수업이 있는 날에는 반드시 체육복과 운동화를 착용하고 등교해야 합니다.
도서관 이용 규칙	도서관은 매일 점심시간과 방과 후 4시까지 이용할 수 있으며, 책을 대출할 때는 학생증이 필요합니다.
' metadata={'source': './학교규칙_예제.txt'}


In [8]:
# 문서 분할기 설정: 각 청크(chunk)의 크기를 300자로 하고, 청크 간 50자 겹침(overlap)을 둠
splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)

# 원본 문서(documents)를 설정한 분할 규칙에 따라 여러 청크로 나눔
docs = splitter.split_documents(documents)

# OpenAI Embeddings 모델을 불러옴 (텍스트를 벡터로 변환)
embeddings = OpenAIEmbeddings()

# 청크로 나눈 문서(docs)를 임베딩하여 Chroma 벡터 데이터베이스에 저장
# persist_directory="./school_db" → 로컬 폴더에 DB를 저장하여 재사용 가능
db = Chroma.from_documents(docs, embeddings, persist_directory="./school_db")

# Chroma DB를 검색용 retriever 객체로 변환 (질문과 유사한 문서 검색에 사용)
retriever = db.as_retriever()

# OpenAI Chat 모델 로드: gpt-3.5-turbo, 온도(창의성) 0.3으로 설정
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3)

# RetrievalQA 체인 생성: retriever로 검색한 문서를 바탕으로 llm이 답변 생성
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)



In [9]:
queries = [
    "중간고사와 기말고사 일정 알려줘.",
    "체육 수업 준비물 규칙 알려줘.",
    "도서관 이용 규칙은 어떻게 돼?",
    "수업 준비물은 어떻게 해야 하나요?",
    "숙제 제출 규칙 요약해줘."
]


In [10]:
for query in queries:
    result = qa_chain.invoke(query)
    print(f"\n[질문]: {query}\n[응답]: {result['result']}")



[질문]: 중간고사와 기말고사 일정 알려줘.
[응답]: 중간고사는 2025년 9월 15일부터 19일까지 진행됩니다. 기말고사는 2025년 12월 10일부터 12월 14일까지 진행됩니다.

[질문]: 체육 수업 준비물 규칙 알려줘.
[응답]: 죄송합니다. 제가 가지고 있는 정보에는 "체육 수업 준비물 규칙"에 대한 내용이 포함되어 있지 않습니다.

[질문]: 도서관 이용 규칙은 어떻게 돼?
[응답]: 도서관은 매일 점심시간과 방과 후 4시까지 이용할 수 있으며, 책을 대출할 때는 학생증이 필요합니다.

[질문]: 수업 준비물은 어떻게 해야 하나요?
[응답]: 수업 준비물로는 교과서, 공책, 필기도구를 매 수업 시간에 반드시 지참해야 합니다.

[질문]: 숙제 제출 규칙 요약해줘.
[응답]: 숙제는 수업 후 다음날 아침 조회시간 전까지 제출해야 하며, 지각 제출 시 지각으로 처리됩니다.
